In [ ]:
from __future__ import division 
from pytesmo.metrics import nrmsd

from math import sqrt
import numpy as np
import random


import pandas as pd
import time
import impyute as impy
import math

from collections import defaultdict
from chefboost import Chefboost as chef
from openpyxl import load_workbook


In [ ]:
def DMI(complete_data , df_incomplete_data):      
    #df_incomplete_data=pd.DataFrame(data=X_incomplete, columns=X_incomplete.columns, index=X_incomplete.index)
    
    ######################################Step-1##############################################
    
    #Split DF into DI and DC subsets
    
    DI = df_incomplete_data[df_incomplete_data.isnull().any(axis=1)] #Xmis is the incomplete subset of X
    index_list_DI = DI.index
    DC = df_incomplete_data.dropna(inplace=False) #Xobs is the complete subset of X
    L = 0
    if len(DC) < 10:
        randomNumbers = []
        for i in range(1,11):
            temp = random.randint(1,len(complete_data))
            if temp not in randomNumbers:
                randomNumbers.append(temp)
        #print(randomNumbers)
        #df.iloc[[2, 3, 4]]
        df_incomplete_data.iloc[randomNumbers] = complete_data.iloc[randomNumbers]
        DI = df_incomplete_data[df_incomplete_data.isnull().any(axis=1)] #Xmis is the incomplete subset of X
        index_list_DI = DI.index
        DC = df_incomplete_data.dropna(inplace=False)
    
    
       
        

    
    ######################################Step-2##############################################
    
    #total no. of attributes in DI having missing values
    M = DI.columns.get_indexer(DI.columns[DI.isnull().any()].tolist())
    
    all_leaves = []
    L = 0
    Ij = {}
    ranges = []

    
    for Ai in range(len(M)):
        DC_CPY = DC.copy()
        index_list = DC_CPY.index
        min_value = min(DC_CPY[DC_CPY.columns[M[Ai]]])
        max_value = max(DC_CPY[DC_CPY.columns[M[Ai]]])
        domain_size = max_value - min_value;
        #NC = int(math.sqrt(domain_size))
        NC = 1
        nc1 = NC
    
        
        min_range = min_value
        max_range = min_value + (domain_size / NC)
        ranges.append([min_range,max_range])
        while NC - 1 != 0:
            min_range = max_range
            max_range = min_range + domain_size / NC
            ranges.append([min_range,max_range])
            NC -= 1
         
        DC_CPY['Decision'] = ['' for _ in range(len(DC_CPY))]
        for row in range(len(DC_CPY)):      
            for rg in ranges:
                if  rg[0] <= DC_CPY.iloc[row, M[Ai]] <= rg[1]:
                    DC_CPY.iloc[row, len(DC_CPY.columns)-1] = str(rg)
                    #print(DC_CPY.iloc[row, len(DC_CPY.columns)-1])
                    break
 
        DC_CPY.drop(DC_CPY.columns[M[Ai]], axis=1, inplace=True)
  
        
        DC_CPY_Temp = DC_CPY.copy()
        
        config = {'algorithm': 'C4.5'} #ID3, C4.5, CART, CHAID or Regression
        model = chef.fit(DC_CPY, config)
        chef.save_model(model, "model.pkl")

        
        modelnew = chef.load_model("model.pkl")
        leaves_one_missing_col = defaultdict(list)
          
        #print("4*")
        for row in range(len(DC_CPY_Temp)):
            prediction = chef.predict(modelnew,DC_CPY_Temp.iloc[row,:-1].tolist())
        
            leaves_one_missing_col[prediction].append(index_list[row])
            Ij[prediction] = False
            
    ######################################Step-3##############################################
    
        DI_temp = DI.drop(DI.columns[M[Ai]], axis=1, inplace=False)
        
        for row in range(len(DI)):
            if pd.isnull(DI.iloc[row, M[Ai]]):
                prediction = chef.predict(modelnew,DI_temp.iloc[row].tolist())
                leaves_one_missing_col[prediction].append(index_list_DI[row])
            
        si = len(leaves_one_missing_col.keys())
        
        L = L + si
                
        all_leaves.append(leaves_one_missing_col)
    print("Length of Leaves",len(all_leaves))

    ######################################Step-4##############################################
    total = 0
    count1 = 0
    #print("all leaves", len(all_leaves))
    #print()
    for tree_leaves in all_leaves:
        for key in tree_leaves:
            dj_indices = tree_leaves[key]
            #print(len(dj_indices))
            dj = df_incomplete_data.loc[dj_indices, :]
            dj.sort_index(inplace=True)
            if dj.isnull().sum().sum() != 0:
                #print(dj.head())
                dj_filled_EMI,itr = impy.em(dj.values)
                #print(dj_filled_EMI)
                df_dj_after_EMI=pd.DataFrame(data = dj_filled_EMI, columns=df_incomplete_data.columns, index=dj.index)  
                #print(df_dj_after_EMI.head())
                
                df_incomplete_data = pd.concat([df_incomplete_data , df_dj_after_EMI])
                
                df_incomplete_data = df_incomplete_data.loc[~df_incomplete_data.index.duplicated(keep='last')]
                
                df_incomplete_data.sort_index(inplace=True)
                total += itr
            count1 += 1
            
           
    #print(df_incomplete_data)  
    #print(count1, total, delta)       
    return df_incomplete_data, count1, total, nc1;




##########################################################################################




In [ ]:

def nrms(all_dataset_imputed,df_complete_data):
    numvar= len(df_complete_data.columns)
    #sum_of_RMSE = 0
    sum_of_NRMSE = 0
    #sum_of_AE = 0
    for i in range(0,numvar):
        
        l1_after_imputation = all_dataset_imputed.iloc[:,i].values     
        l2_complete_data = df_complete_data.iloc[:,i].values     
        nrms = nrmsd(l2_complete_data, l1_after_imputation) #Normalized root-mean-square deviation (nRMSD)
        sum_of_NRMSE = sum_of_NRMSE + nrms
    average_NRMSE_All_Cols = sum_of_NRMSE / numvar
    print(average_NRMSE_All_Cols)
    return average_NRMSE_All_Cols




In [ ]:
def fileName(indexFolder,cFilePath, fileName,count,col):
    summaryName = "Table-NRMS.xlsx"
    wb = load_workbook(summaryName)
    ws1 = wb.active
    percentageList = [20]
    for j  in percentageList:
        #col = cFilePath.columns
        incompleteFileName = "Data_" + str(indexFolder) + "_" + fileName + "_" + str(j) + "%.csv"
        df = pd.read_csv("C:\\Users\\patel\\Desktop\\DM\\Incomplete datasets(1)\\Data " + str(indexFolder) + "\\"+incompleteFileName, header = None)
        df.columns = col
        
        df.drop(columns = ['Labels'], axis = 1,  inplace = True)
        #col1 = cFilePath.columns
        start_time = time.time()
        all_dataset_imputed, count1, total, nc = DMI(cFilePath , df)
        
        
        timeFinal = (time.time() - start_time)
        all_dataset_imputed.to_csv("C:\\Users\\patel\\Desktop\DM\\New folder\\Data 14\\" + incompleteFileName, columns = cFilePath.columns, index = None)

        
        nrmsValue = nrms(all_dataset_imputed,cFilePath)
       
        ws1['B'+str(count)] = nrmsValue
        ws1['C'+str(count)] = timeFinal
        print(nrmsValue,timeFinal,incompleteFileName + ", Number of Horizontal Partitions: " + str(count1) + ", EM Total iterations: " + str(total)  + ", Convergence(%): 10%")
        ws1['D'+str(count)] = ("Number of Horizontal Partitions: " + str(count1) + ", EM iterations: " + str(total)  + ", Convergence(%): 10%" + ", NC: " + str(nc))

        count += 1
        #print("iteration",itr)


        wb.save(summaryName) 
    return count

In [ ]:
folderName = "Data " + str(7) 
completeFileName = 'Data_' + str(7) + '.csv'
cFilePath = pd.read_csv("C:\\Users\\patel\\Desktop\\DM\\Complete datasets(1)\\"+completeFileName)
col = cFilePath.columns
cFilePath.drop(columns = ['Labels'],axis = 1, inplace = True)
df_complete_data=pd.DataFrame(data=cFilePath, columns=cFilePath.columns, index=cFilePath.index)
    
#count = 268
count1 = fileName(7,cFilePath, "AE", count,col)
count2 = fileName(7,cFilePath, "AG", count1,col)
count3 = fileName(7,cFilePath, "AL", count2)
count4 = fileName(7,cFilePath, "AN", count3,col) 
count5 = fileName(7,cFilePath, "AW", count4)
count6 = fileName(7,cFilePath, "C", count5)
count7 = fileName(7,cFilePath, "NE", count6)
count8 = fileName(7,cFilePath, "NG", count7)
count9 = fileName(7,cFilePath, "NL", count8)
count10 = fileName(7,cFilePath, "NN", count9)
count11 = fileName(7,cFilePath, "NW", count10)